In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#### 定义模型

包括各层参数以及前向函数forwad()，后向函数backward()将自动生成。

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # y = Wx + b
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


可学习的参数通过net.parameters()返回。

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


#### 损失函数

In [5]:
input = torch.randn(1, 1, 32, 32) # 随机输入值
output = net(input) # 正向
print(output)

tensor([[ 0.0171,  0.0345, -0.1835,  0.0492,  0.1534, -0.0464, -0.0731,  0.0048,
          0.1161, -0.0566]], grad_fn=<AddmmBackward>)


In [6]:
target = torch.randn(10)  # 随机目标值
target = target.view(1, -1)  # 调整为output同样的形状

criterion = nn.MSELoss() # 均方误差
loss = criterion(output, target)
print(loss)

tensor(0.8530, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
net.zero_grad()   # 所有参数的gradient buffers清零
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward() # 后向传播一次
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0118, -0.0269, -0.0054,  0.0046,  0.0083,  0.0206])


#### 更新参数

使用SGD更新：weight = weight - learning_rate * gradient

In [10]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()   # 清零

# 更新循环：
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()